In [1]:
import numpy as np

In [2]:
def calc_mse(y, y_pred):
    err = np.mean((y - y_pred)**2)
    return err

In [5]:
X = np.array([[ 1,  1],
              [ 1,  1],
              [ 1,  2],
              [ 1,  5],
              [ 1,  3],
              [ 1,  0],
              [ 1,  5],
              [ 1, 10],
              [ 1,  1],
              [ 1,  2]])
y = [45, 55, 50, 55, 60, 35, 75, 80, 50, 60]

In [51]:
n = X.shape[0]

eta = 1e-2 
n_iter = 1000 #ИЗМЕНЕНИЕ
epsilon = 1e-4 #ИЗМЕНЕНИЕ

W = np.array([1, 0.5])
print(f'Number of objects = {n} \
       \nLearning rate = {eta} \
       \nInitial weights = {W} \n')

for i in range(n_iter):
    y_pred = np.dot(X, W)
    if (i > 0) and (np.abs(np.linalg.norm(y_pred) - np.linalg.norm(y_pred_old)) < epsilon): #ИЗМЕНЕНИЕ
        break #ИЗМЕНЕНИЕ
    err = calc_mse(y, y_pred)
#    for k in range(W.shape[0]): #ИЗМЕНЕНИЕ
    W -= eta * (1/n * 2 * X.T @ (y_pred - y)) #ИЗМЕНЕНИЕ
    if i % 10 == 0:
        eta /= 1.1
        print(f'Iteration #{i}: W_new = {W}, MSE = {round(err, 2)}')
    y_pred_old = y_pred #ИЗМЕНЕНИЕ

Number of objects = 10        
Learning rate = 0.01        
Initial weights = [1.  0.5] 

Iteration #0: W_new = [2.08 4.27], MSE = 3047.75
Iteration #10: W_new = [ 6.67106886 10.61676385], MSE = 749.71
Iteration #20: W_new = [ 9.49320908 10.25731657], MSE = 648.91
Iteration #30: W_new = [11.85740092  9.83349244], MSE = 570.46
Iteration #40: W_new = [13.86876921  9.46898661], MSE = 508.03
Iteration #50: W_new = [15.59085668  9.15672679], MSE = 457.73
Iteration #60: W_new = [17.07337653  8.88789585], MSE = 416.77
Iteration #70: W_new = [18.35601294  8.65530964], MSE = 383.06
Iteration #80: W_new = [19.47073522  8.45317196], MSE = 355.08
Iteration #90: W_new = [20.44350656  8.27677488], MSE = 331.65
Iteration #100: W_new = [21.29557245  8.12226587], MSE = 311.9
Iteration #110: W_new = [22.044442    7.98646992], MSE = 295.12
Iteration #120: W_new = [22.7046421   7.86675281], MSE = 280.78
Iteration #130: W_new = [23.2883015   7.76091518], MSE = 268.46
Iteration #140: W_new = [23.80560705  7